In [ ]:
# process matlab produced csv and convert pickle back to csv
# Conny Lin | June 5, 2020

In [6]:
# load library
import os
import sys
import pickle
import time
import numpy as np
import pandas as pd
import subprocess
import glob

# import functions
dir_function_collection = [
'/Users/connylin/Dropbox/Code/language/python_lib/localpackage/mwt',
'/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone/brainstation_capstone/0_lib'
]
for p in dir_function_collection:
    sys.path.insert(1, p)

import BrainStationLib as bs

In [17]:
# define global varialbes/paths
pDropbox_home = '/Users/connylin/Dropbox/MWT/db'
pCapstone = '/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone/data'

In [45]:
# load MWTDB for dropbox
MWTDB = pd.read_csv(pDropbox_home+'/MWTDB.csv', index_col=False)
MWTDB

,mwtid,mwtpath,expname,groupname,platename
0,0,/Users/connylin/Dropbox/MWT/db/MWT/20141216B_S...,20141216B_SI_100s30x10s10s_MT1,RB982,20141216_175807
1,1,/Users/connylin/Dropbox/MWT/db/MWT/20141216B_S...,20141216B_SI_100s30x10s10s_MT1,RB982,20141216_173721
2,2,/Users/connylin/Dropbox/MWT/db/MWT/20141216B_S...,20141216B_SI_100s30x10s10s_MT1,RB982,20141216_182015
3,3,/Users/connylin/Dropbox/MWT/db/MWT/20141216B_S...,20141216B_SI_100s30x10s10s_MT1,RB982,20141216_171635
4,4,/Users/connylin/Dropbox/MWT/db/MWT/20141216B_S...,20141216B_SI_100s30x10s10s_MT1,RB982_400mM,20141216_180658
...,...,...,...,...,...
7289,7289,/Users/connylin/Dropbox/MWT/db/MWT to check/20...,20140819C_SM_100s30x10s10s,ungroup,20140819_172227
7290,7290,/Users/connylin/Dropbox/MWT/db/MWT to check/20...,20140819C_SM_100s30x10s10s,ungroup,20140819_191228
7291,7291,/Users/connylin/Dropbox/MWT/db/MWT to check/20...,20140819C_SM_100s30x10s10s,ungroup,20140819_182649
7292,7292,/Users/connylin/Dropbox/MWT/db/MWT to check/20...,20140819C_SM_100s30x10s10s,ungroup,20140819_173055


In [16]:
pickle.load(open(ptrinity_csv_list[0],'rb'))

,mwtid_trdb,etoh,time,speed,bias,tap,loc_x,loc_y,morphwidth,midline,area,angular,aspect,kink,curve,crab,wormid
1,477,1,1583.182,0.1404,0.0,0.0,17.3135,50.2187,0.0975,1.0216,0.115911,7.7,0.250,16.9,31.3,0.0127,8800
2,477,1,1583.254,0.0684,0.0,0.0,17.3153,50.2169,0.0978,0.9616,0.115911,2.2,0.271,21.1,29.1,0.0045,8800
3,477,1,1583.301,0.0520,0.0,0.0,17.3184,50.2129,0.0986,0.9668,0.114453,2.1,0.262,24.3,26.9,0.0030,8800
4,477,1,1583.381,0.0324,0.0,0.0,17.3188,50.2131,0.1000,0.9443,0.114453,5.8,0.243,21.2,34.4,0.0082,8800
5,477,1,1583.464,0.0540,0.0,0.0,17.3229,50.2100,0.1041,0.9677,0.121743,0.6,0.259,9.7,37.1,0.0038,8800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2234864,477,1,1387.721,0.0913,1.0,0.0,14.4053,36.8805,0.1199,0.8329,0.115182,7.6,0.175,48.4,14.0,0.0094,7714
2234865,477,1,1387.795,0.1089,1.0,0.0,14.4017,36.8816,0.1206,0.8295,0.116640,7.8,0.172,47.8,14.9,0.0044,7714
2234866,477,1,1387.841,0.1855,1.0,0.0,14.3945,36.8830,0.1200,0.8411,0.118098,12.8,0.182,50.0,14.5,0.0020,7714
2234867,477,1,1387.881,0.1725,0.0,0.0,14.3850,36.8857,0.1181,0.8335,0.116640,11.8,0.190,55.1,18.1,0.0040,7714


In [19]:
MWTDBTNT = pickle.load(open(os.path.join(pCapstone,'MWTDB_trinity_N2400mM.pickle'),'rb'))

In [ ]:
istart = 5
for iMWTDB in MWTDB.index.values[istart:]:
    # get path
    pmwt = MWTDB.loc[iMWTDB, 'mwtpath']
    print(f'{iMWTDB}: {pmwt}')

    # check if has trinity_cleaned.csv, 
    ptrinity_cleaned = os.path.join(pmwt,'trinity_cleaned.csv')
    ptrinity_pickle = os.path.join(pmwt,'trinity_all_worms.pickle')
    ptrinity_idin = os.path.join(pmwt,'trinity_idin.csv')
    ptrinity_meta = os.path.join(pmwt,'trinity_meta.csv')
    ptrinity_csv = os.path.join(pmwt,'trinity.csv')
    
    # instantiate dataframe
    df = pd.DataFrame({'failed':[0]})

    # if no, trinity_cleaned.csv
    if not os.path.isfile(ptrinity_cleaned):
        # check if has trinity_all_worms.pickle
        # if yes
        if os.path.isfile(ptrinity_pickle):
            assert False, 'code this option'
            # load pickle file
            # find mwtpath from MWTDBTNT 
            # find matching path from MWTDB
            # get MWTDB id
            # add mwtid column
            # drop mwtid_trdb column
            # save as trinity_cleaned.csv

        # if no, 
        elif not os.path.isfile(ptrinity_pickle):
            # check if have trinity_idin.csv
            # if no, 
            if not os.path.isfile(ptrinity_idin):
                # check if has both trinity_meta.csv and trinity.csv
                # if yes, 
                if os.path.isfile(ptrinity_meta) & \
                    os.path.isfile(ptrinity_csv):
                    assert False, 'code this option'
                    # load trinity_meta.csv
                    # create worm id column by rows
                    # load trinity.csv
                    # concat with trinity.csv, 
                    # remove columns
                # if no, leave the plate be
                else:
                    continue
                    assert False, 'code this option'
                    
            # if yes,
            elif os.path.isfile(ptrinity_idin):
                # load trinity_idin.csv
                df = pd.read_csv(ptrinity_idin)
                TD = bs.TrinityData(ptrinity_idin)
                df = TD.loaddata()
            else:
                
                assert False, 'code this option'
            
            # [now should have the same dataframe]
            # check df is right size
            if df.shape[1]==14:
                # clean out nan values
                df.dropna(axis=0, inplace=True)
                # add plate id
                plateid = MWTDB.loc[iMWTDB, 'mwtid']
                df.insert(0, 'mwtid', np.tile(plateid, df.shape[0]))
                # add etoh id
                etoh = 0
                if '00mM' in MWTDB.loc[iMWTDB, 'groupname']:
                    etoh = 1
                df.insert(1, 'etoh', np.tile(etoh, df.shape[0]))
                # save as trinity_cleaned.csv 
                df.to_csv(ptrinity_cleaned, index=False)
                # delete df to avoid writing wrong df files
                del df

        else:
            assert False, 'code this option'

    # final check to see if trinity_cleaned exist
    if os.path.isfile(ptrinity_cleaned):
        print(f'\ttrinity_cleaned.csv succeed, delete redundant files')
        # if so, delete trinity_meta.csv, trinity.csv, and pickle
        if os.path.isfile(ptrinity_idin):
            os.remove(ptrinity_idin)
            print('\tdeleted trinity_idin.csv')
        if os.path.isfile(ptrinity_pickle):
            os.remove(ptrinity_pickle)
            print('\tdeleted trinity pickle file')
        if os.path.isfile(ptrinity_meta):
            os.remove(ptrinity_meta)
            print('\tdeleted trinity meta file')
        if os.path.isfile(ptrinity_csv):
            os.remove(ptrinity_csv)
            print('\tdeleted trinity.csv file')

5: /Users/connylin/Dropbox/MWT/db/MWT/20141216B_SI_100s30x10s10s_MT1/RB982_400mM/20141216_183020
6: /Users/connylin/Dropbox/MWT/db/MWT/20141216B_SI_100s30x10s10s_MT1/RB982_400mM/20141216_174911
	trinity_cleaned.csv succeed, delete redundant files
	deleted trinity_idin.csv
7: /Users/connylin/Dropbox/MWT/db/MWT/20141216B_SI_100s30x10s10s_MT1/RB982_400mM/20141216_172726
	trinity_cleaned.csv succeed, delete redundant files
	deleted trinity_idin.csv
8: /Users/connylin/Dropbox/MWT/db/MWT/20141216B_SI_100s30x10s10s_MT1/VC2016/20141216_183817
	trinity_cleaned.csv succeed, delete redundant files
	deleted trinity_idin.csv
9: /Users/connylin/Dropbox/MWT/db/MWT/20141216B_SI_100s30x10s10s_MT1/VC2016/20141216_185718
	trinity_cleaned.csv succeed, delete redundant files
	deleted trinity_idin.csv
10: /Users/connylin/Dropbox/MWT/db/MWT/20141216B_SI_100s30x10s10s_MT1/VC2016/20141216_191806
	trinity_cleaned.csv succeed, delete redundant files
	deleted trinity_idin.csv
11: /Users/connylin/Dropbox/MWT/db/MW

In [95]:
df.shape[1]==14

True

In [ ]:
assert False, 'stop'